## **NLP - Text Classification Project**
Group H - August 2025

Classification of tweets from Twitter that have been manually tagged for sentiment analysis.

In [1]:
# # Needed for Google Colab
# !pip install --quiet evaluate transformers optuna datasets nltk scikit-learn
# !pip install numpy==1.26.4

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

from wordcloud import WordCloud, STOPWORDS
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset
from optuna.pruners import MedianPruner
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import string
import time
import glob
import nltk
import evaluate
import transformers
import torch
import optuna
import wandb
wandb.login()

os.environ["WANDB_PROJECT"] = "tweet-sentiment-classification_split_to_test"
os.environ["WANDB_INIT_TIMEOUT"] = "180"

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

env: CUDA_LAUNCH_BLOCKING=1
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


wandb: Currently logged in as: hillas (hillas-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
# Load data
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_train.csv', encoding='latin1')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_test.csv', encoding='latin1')

In [4]:
# Merge and shuffle for better stratified splits
df_full = pd.concat([df_train, df_test], ignore_index=True)
df_full = df_full.sample(frac=1.0, random_state=42).reset_index(drop=True)

### Pre-processing the Data

The tweets were cleaned by lowercasing (to reduce redundancy), removing stopwords, punctuation, numbers, short words, and applying lemmatization to reduce words to their base form (e.g. running → run). <br>This helps reduce noise and improve model performance.

In [5]:
# # Try without pre-processing
# is_preprocessed = "no_preprocess"
# df_full["clean_text"] = df_full["OriginalTweet"]

In [6]:
# Try minimal pre-processing
is_preprocessed = "minimal_preprocess"
def light_preprocess(text):
    # text = re.sub(r"http\S+|www.\S+", "", text)   # Remove URLs
    # text = re.sub(r"@\w+", "", text)              # Remove @mentions
    # text = re.sub(r"#(\w+)", r"\1", text)         # Unwrap hashtags
    return text.strip()                             # Remove unnecessary spaces
df_full["clean_text"] = df_full["OriginalTweet"].apply(light_preprocess)

## Fine-Tuning Pretrained Language Models

Apply NLP techniques using transfer learning on our tweets dataset. Specifically, fine-tuning two pretrained transformer-based models from the Hugging Face library — BERT and RoBERTa — on our sentiment classification task. These models will be trained using both standard PyTorch and the Hugging Face API. Model performance will be monitored and tuned using hyperparameter optimization (Optuna) and experiment tracking (Weights & Biases).

**Load Pretrained Models**

Initialize tokenizers and models for BERT and RoBERTa, both widely used transformer architectures for text classification. The classification head is configured based on the number of sentiment labels.

In [7]:
# Choose pretrained models
bert_model_name = "bert-base-uncased"
roberta_model_name = "roberta-base"

sentiment_labels = df_full['Sentiment'].unique()
n_labels = len(sentiment_labels)

# Load BERT tokenizer and model
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=n_labels)

# Load RoBERTa tokenizer and model
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name, num_labels=n_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight

**Encode Sentiment Labels**

Map each unique sentiment label to a numeric ID for model compatibility, and apply this mapping to both training and validation datasets.

In [8]:
# Mapping sentiments to unique numeric IDs
unique_labels = sorted(df_full["Sentiment"].unique())
label2id = {label: idx for idx, label in enumerate(unique_labels)}
df_full["label"] = df_full["Sentiment"].map(label2id)

In [9]:
# Stratified split: 70% train, 15% val, 15% test
train_val_df, test_df = train_test_split(df_full, test_size=0.15, stratify=df_full["label"], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.1765, stratify=train_val_df["label"], random_state=42)

# Confirm sizes
print("Train size:", len(train_df))
print("Val size:", len(val_df))
print("Test size:", len(test_df))

Train size: 31466
Val size: 6745
Test size: 6744


**Use Small Subsets for Quick Evaluation**

Select shuffled samples from each training and validation dataset for both BERT and RoBERTa. This allows faster experimentation during model development.

In [10]:
train_subset_df, _ = train_test_split(
    train_df[["clean_text", "label"]],
    train_size=2000,
    stratify=train_df["label"],
    random_state=42
)

val_subset_df, _ = train_test_split(
    val_df[["clean_text", "label"]],
    train_size=500,
    stratify=val_df["label"],
    random_state=42
)

**Tokenization**

Define tokenization functions for BERT and RoBERTa to preprocess text with truncation and fixed padding.

Transform training and validation DataFrames into Dataset objects compatible with Hugging Face workflows.

Apply tokenization to training and validation datasets using each model's tokenizer.

In [11]:
# Tokenize function
def tokenize_function_bert(examples):
    return bert_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=64)

def tokenize_function_roberta(examples):
    return roberta_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=64)

In [12]:
# Convert DataFrame to Hugging Face Dataset
hf_subset_train = Dataset.from_pandas(train_subset_df)
hf_subset_val = Dataset.from_pandas(val_subset_df)

hf_train = Dataset.from_pandas(train_df[["clean_text", "label"]])
hf_val = Dataset.from_pandas(val_df[["clean_text", "label"]])
hf_test = Dataset.from_pandas(test_df[["clean_text", "label"]])

In [13]:
# Tokenize subsets
# Tokenize for BERT
tokenized_bert_train_sub = hf_subset_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val_sub = hf_subset_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Tokenize for RoBERTa
tokenized_roberta_train_sub = hf_subset_train.map(tokenize_function_roberta, batched=True)
tokenized_roberta_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_val_sub = hf_subset_val.map(tokenize_function_roberta, batched=True)
tokenized_roberta_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [14]:
# Tokenize full dataset
# Tokenize for BERT
tokenized_bert_train = hf_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val = hf_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_test = hf_test.map(tokenize_function_bert, batched=True)
tokenized_bert_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Tokenize for RoBERTa
tokenized_roberta_train = hf_train.map(tokenize_function_roberta, batched=True)
tokenized_roberta_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_val = hf_val.map(tokenize_function_roberta, batched=True)
tokenized_roberta_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_test = hf_test.map(tokenize_function_roberta, batched=True)
tokenized_roberta_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/31466 [00:00<?, ? examples/s]

Map:   0%|          | 0/6745 [00:00<?, ? examples/s]

Map:   0%|          | 0/6744 [00:00<?, ? examples/s]

Map:   0%|          | 0/31466 [00:00<?, ? examples/s]

Map:   0%|          | 0/6745 [00:00<?, ? examples/s]

Map:   0%|          | 0/6744 [00:00<?, ? examples/s]

In [15]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"]
    }

**Hyperparameters Tuning with Optuna**

In [16]:
def build_trainer(model_checkpoint, trial, run_prefix, train_dataset, val_dataset):
    # Sample hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_train_epochs", 2, 5)
    n_samples = len(train_dataset)

    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    run_name = f"{run_prefix}-ep{num_epochs}-lr{learning_rate}-bs{batch_size}-samples{n_samples}-run{int(time.time())}-{is_preprocessed}"

    args = TrainingArguments(
        output_dir=f"./results/{run_prefix}/{run_name}",
        disable_tqdm=True,
        fp16=True,
        eval_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        weight_decay=0.01,
        load_best_model_at_end=True,
        save_total_limit=1,
        logging_strategy="epoch",
        logging_dir=f"./logs/{run_prefix}/{run_name}",
        run_name=run_name,
        report_to="wandb",
        metric_for_best_model="f1_macro",
        greater_is_better=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    return trainer

In [17]:
def objective_bert(trial):
    trainer = build_trainer(
        model_checkpoint="bert-base-uncased",
        trial=trial,
        run_prefix="bert",
        train_dataset=tokenized_bert_train_sub,
        val_dataset=tokenized_bert_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1_macro"]

def objective_roberta(trial):
    trainer = build_trainer(
        model_checkpoint="roberta-base",
        trial=trial,
        run_prefix="roberta",
        train_dataset=tokenized_roberta_train_sub,
        val_dataset=tokenized_roberta_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()

    return eval_result["eval_f1_macro"]

In [18]:
study_bert = optuna.create_study(direction="maximize",
                                 pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                 study_name=f"bert_study_stratify_{is_preprocessed}",
                                 storage=f"sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/bert_study_stratify_{is_preprocessed}.db",
                                 load_if_exists=True)
study_bert.optimize(objective_bert, n_trials=5)
wandb.finish()

[I 2025-07-30 21:14:20,101] A new study created in RDB with name: bert_study_stratify_minimal_preprocess
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1416991230.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.5774, 'grad_norm': 4.061391353607178, 'learning_rate': 2.1729204886873087e-05, 'epoch': 1.0}
{'eval_loss': 1.52866792678833, 'eval_accuracy': 0.316, 'eval_f1_macro': 0.20381664407698566, 'eval_runtime': 0.4298, 'eval_samples_per_second': 1163.219, 'eval_steps_per_second': 37.223, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.3802, 'grad_norm': 7.989249229431152, 'learning_rate': 1.6404261396809323e-05, 'epoch': 2.0}
{'eval_loss': 1.2827919721603394, 'eval_accuracy': 0.416, 'eval_f1_macro': 0.38911502561703554, 'eval_runtime': 0.4328, 'eval_samples_per_second': 1155.326, 'eval_steps_per_second': 36.97, 'epoch': 2.0}


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.9356, 'grad_norm': 7.612660884857178, 'learning_rate': 1.6211858429574552e-07, 'epoch': 5.0}
{'eval_loss': 1.197390079498291, 'eval_accuracy': 0.48, 'eval_f1_macro': 0.49043757078994304, 'eval_runtime': 0.4372, 'eval_samples_per_second': 1143.742, 'eval_steps_per_second': 36.6, 'epoch': 5.0}
{'train_runtime': 39.3899, 'train_samples_per_second': 253.872, 'train_steps_per_second': 7.997, 'train_loss': 1.2256096249534971, 'epoch': 5.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
[I 2025-07-30 21:18:00,309] Trial 4 finished with value: 0.49043757078994304 and parameters: {'learning_rate': 1.7022451351053277e-05, 'batch_size': 32, 'num_train_epochs': 5}. Best is trial 3 with value: 0.5623185946011807.


{'eval_loss': 1.197390079498291, 'eval_accuracy': 0.48, 'eval_f1_macro': 0.49043757078994304, 'eval_runtime': 0.447, 'eval_samples_per_second': 1118.598, 'eval_steps_per_second': 35.795, 'epoch': 5.0}


eval/accuracy,▁▄▆▆▇▇▂▃▄▄▁▄▄▃▅▇▇██▁▃▅▅▆▆
eval/f1_macro,▁▅▆▇▇▇▂▄▄▄▂▅▅▄▆▇███▂▃▅▆▇▇
eval/loss,█▄▂▂▂▂▇▆▅▅█▆▆▆▃▁▁▁▁▇▅▃▃▃▃
eval/runtime,▁▁▁▁▁▁▃▃▃▃███▁▁▁▁▁▁▁▁▁▁▁▁
eval/samples_per_second,█▇███▇▄▄▄▄▁▁▁██████████▇▇
eval/steps_per_second,▂▂▃▃▃▁▆▇▇▇██▇▃▃▃▃▃▂▃▃▂▂▂▁
train/epoch,▁▁▃▃▅▆▆███▁▁▃▃▅▅▅▁▃▃▃▁▁▃▃▅▆▆██▁▁▃▃▅▆▆███
train/global_step,▁▁▂▂▃▄▄▅▅▅▂▂▄▄▆▆▆▄███▁▁▂▂▃▄▄▅▅▅▁▁▂▂▃▄▅▅▅
train/grad_norm,▁▃█▇▄▂▂▄▂▄▁▄▅▇▅▂▅█▄▃
train/learning_rate,█▆▅▃▁▃▂▁▃▁▇▆▄▃▁▅▄▃▂▁
train/loss,█▆▄▂▁█▇▆█▇█▆▃▂▁█▇▅▄▃


In [19]:
study_roberta = optuna.create_study(direction="maximize",
                                    pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                    study_name=f"roberta_study_stratify_{is_preprocessed}",
                                    storage=f"sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/roberta_study_stratify_{is_preprocessed}.db",
                                    load_if_exists=True)
study_roberta.optimize(objective_roberta, n_trials=5)
wandb.finish()

[I 2025-07-30 21:18:03,229] A new study created in RDB with name: roberta_study_stratify_minimal_preprocess
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1416991230.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.5107, 'grad_norm': 10.67564582824707, 'learning_rate': 3.452340426579314e-05, 'epoch': 1.0}
{'eval_loss': 1.2914682626724243, 'eval_accuracy': 0.434, 'eval_f1_macro': 0.3944860657838186, 'eval_runtime': 0.4215, 'eval_samples_per_second': 1186.234, 'eval_steps_per_second': 37.959, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.189, 'grad_norm': 16.504610061645508, 'learning_rate': 2.331685418998594e-05, 'epoch': 2.0}
{'eval_loss': 1.159412145614624, 'eval_accuracy': 0.524, 'eval_f1_macro': 0.5282124573907068, 'eval_runtime': 0.4454, 'eval_samples_per_second': 1122.513, 'eval_steps_per_second': 35.92, 'epoch': 2.0}


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.7196, 'grad_norm': 13.408093452453613, 'learning_rate': 2.5462043482683685e-07, 'epoch': 5.0}
{'eval_loss': 1.2174280881881714, 'eval_accuracy': 0.512, 'eval_f1_macro': 0.5268278249528249, 'eval_runtime': 0.4292, 'eval_samples_per_second': 1164.847, 'eval_steps_per_second': 37.275, 'epoch': 5.0}
{'train_runtime': 42.9637, 'train_samples_per_second': 232.755, 'train_steps_per_second': 7.332, 'train_loss': 1.0901095314631386, 'epoch': 5.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
[I 2025-07-30 21:22:03,005] Trial 4 finished with value: 0.5268278249528249 and parameters: {'learning_rate': 2.6735145656817866e-05, 'batch_size': 32, 'num_train_epochs': 5}. Best is trial 0 with value: 0.5910379517916615.


{'eval_loss': 1.2174280881881714, 'eval_accuracy': 0.512, 'eval_f1_macro': 0.5268278249528249, 'eval_runtime': 0.4506, 'eval_samples_per_second': 1109.715, 'eval_steps_per_second': 35.511, 'epoch': 5.0}


eval/accuracy,▄▆▆██▄▆▆▄▆▆▂▆▇██▁▄▅▆▆▆
eval/f1_macro,▃▇▇██▃▇▇▄▇▇▂▆▇██▁▅▅▆▆▆
eval/loss,▅▂▂▁▁▃▁▁▃▁▁▆▂▂▂▂█▄▃▃▄▄
eval/runtime,▁▁▁▁▁██████▃▃▃▃▄▁▁▁▁▁▁
eval/samples_per_second,█▇███▁▁▁▁▁▁▄▄▄▄▃█████▇
eval/steps_per_second,▃▁▂▁▂█▇██▇█▆▆▇▆▅▂▂▂▂▂▁
train/epoch,▁▁▃▃▅▅▆▆▆▆▁▃▃▃▃▁▁▃▃▃▁▁▃▃▅▅▆▆▆▆▁▃▃▅▅▆▆███
train/global_step,▁▁▂▂▃▃▄▄▄▄▄████▄▄███▂▂▄▄▆▆████▁▂▂▃▃▄▄▅▅▅
train/grad_norm,▂▃▃▃█▆▂▅▂▂▃▅▁▂▇▄▂
train/learning_rate,█▆▃▁▅▁▅▁█▆▃▁▅▄▃▂▁
train/loss,█▅▃▂▇▄▇▄█▅▃▁█▆▄▃▂


In [20]:
best_trial_bert = study_bert.best_trial
print('Bert best trial on subset:')
print(best_trial_bert.params)
best_trial_roberta = study_roberta.best_trial
print('RoBerta best trial on subset:')
print(best_trial_roberta.params)

Bert best trial on subset:
{'learning_rate': 2.3801360227277252e-05, 'batch_size': 32, 'num_train_epochs': 5}
RoBerta best trial on subset:
{'learning_rate': 4.554920353392603e-05, 'batch_size': 32, 'num_train_epochs': 4}


In [21]:
best_params_bert = best_trial_bert.params
run_name_bert = f"bert_final_stratify_{is_preprocessed}-ep{best_params_bert['num_train_epochs']}-lr{best_params_bert['learning_rate']:.1e}-bs{best_params_bert['batch_size']}"
wandb.init(project="tweet-sentiment-classification_split_to_test", name=run_name_bert, reinit=True)

final_trainer_bert = build_trainer(
    model_checkpoint="bert-base-uncased",
    trial=best_trial_bert,
    run_prefix=f"bert_final_stratify_{is_preprocessed}",
    train_dataset=tokenized_bert_train,
    val_dataset=tokenized_bert_val
)
final_trainer_bert.train()
final_trainer_bert.evaluate(tokenized_bert_test)
wandb.finish()

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1416991230.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.862, 'grad_norm': 29.985370635986328, 'learning_rate': 1.905560120635063e-05, 'epoch': 1.0}
{'eval_loss': 0.6107035875320435, 'eval_accuracy': 0.7669384729429207, 'eval_f1_macro': 0.7735242464060336, 'eval_runtime': 5.402, 'eval_samples_per_second': 1248.605, 'eval_steps_per_second': 39.059, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.4516, 'grad_norm': 8.333763122558594, 'learning_rate': 1.4300166835738121e-05, 'epoch': 2.0}
{'eval_loss': 0.4731937348842621, 'eval_accuracy': 0.8306893995552261, 'eval_f1_macro': 0.8372507497705153, 'eval_runtime': 5.453, 'eval_samples_per_second': 1236.944, 'eval_steps_per_second': 38.695, 'epoch': 2.0}


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.1579, 'grad_norm': 33.07847213745117, 'learning_rate': 2.4188374214712656e-08, 'epoch': 5.0}
{'eval_loss': 0.515195906162262, 'eval_accuracy': 0.8604892512972572, 'eval_f1_macro': 0.8646459410051675, 'eval_runtime': 5.3756, 'eval_samples_per_second': 1254.732, 'eval_steps_per_second': 39.251, 'epoch': 5.0}
{'train_runtime': 471.1615, 'train_samples_per_second': 333.919, 'train_steps_per_second': 10.442, 'train_loss': 0.39892827475943216, 'epoch': 5.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.5530809760093689, 'eval_accuracy': 0.8558718861209964, 'eval_f1_macro': 0.859383840081767, 'eval_runtime': 5.4191, 'eval_samples_per_second': 1244.498, 'eval_steps_per_second': 38.937, 'epoch': 5.0}


eval/accuracy,▁▆▆▇██
eval/f1_macro,▁▆▆▇██
eval/loss,█▁▂▃▃▅
eval/runtime,▂▄█▄▁▃
eval/samples_per_second,▇▅▁▅█▆
eval/steps_per_second,▇▅▁▅█▆
train/epoch,▁▁▃▃▅▅▆▆████
train/global_step,▁▁▃▃▅▅▆▆████
train/grad_norm,▇▁▄▃█
train/learning_rate,█▆▄▃▁
train/loss,█▄▂▂▁


In [22]:
best_params_roberta = best_trial_roberta.params
run_name_roberta = f"roberta_final_stratify_{is_preprocessed}-ep{best_params_roberta['num_train_epochs']}-lr{best_params_roberta['learning_rate']:.1e}-bs{best_params_roberta['batch_size']}"
wandb.init(project="tweet-sentiment-classification_split_to_test", name=run_name_roberta, reinit=True)

final_trainer_roberta = build_trainer(
    model_checkpoint="roberta-base",
    trial=best_trial_roberta,
    run_prefix=f"roberta_final_stratify_{is_preprocessed}",
    train_dataset=tokenized_roberta_train,
    val_dataset=tokenized_roberta_val
)
final_trainer_roberta.train()
final_trainer_roberta.evaluate(tokenized_roberta_test)
wandb.finish()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1416991230.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.9378, 'grad_norm': 13.393064498901367, 'learning_rate': 3.4196620031186844e-05, 'epoch': 1.0}
{'eval_loss': 0.7695580124855042, 'eval_accuracy': 0.6951816160118607, 'eval_f1_macro': 0.7056358032149983, 'eval_runtime': 5.5245, 'eval_samples_per_second': 1220.918, 'eval_steps_per_second': 38.193, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.5545, 'grad_norm': 13.495407104492188, 'learning_rate': 2.2832464068200213e-05, 'epoch': 2.0}
{'eval_loss': 0.5419302582740784, 'eval_accuracy': 0.8060785767234989, 'eval_f1_macro': 0.8124322702129481, 'eval_runtime': 5.5776, 'eval_samples_per_second': 1209.311, 'eval_steps_per_second': 37.83, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.2692, 'grad_norm': 12.716752052307129, 'learning_rate': 5.786230123720278e-08, 'epoch': 4.0}
{'eval_loss': 0.5578931570053101, 'eval_accuracy': 0.8297998517420311, 'eval_f1_macro': 0.8356663762092833, 'eval_runtime': 5.437, 'eval_samples_per_second': 1240.568, 'eval_steps_per_second': 38.808, 'epoch': 4.0}
{'train_runtime': 381.9509, 'train_samples_per_second': 329.529, 'train_steps_per_second': 10.305, 'train_loss': 0.5380064413799504, 'epoch': 4.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.6006706357002258, 'eval_accuracy': 0.8232502965599051, 'eval_f1_macro': 0.8282388040749037, 'eval_runtime': 5.3839, 'eval_samples_per_second': 1252.62, 'eval_steps_per_second': 39.191, 'epoch': 4.0}


eval/accuracy,▁▇▇██
eval/f1_macro,▁▇▇██
eval/loss,█▁▁▂▃
eval/runtime,▆█▄▃▁
eval/samples_per_second,▃▁▅▆█
eval/steps_per_second,▃▁▅▆█
train/epoch,▁▁▃▃▆▆████
train/global_step,▁▁▃▃▆▆████
train/grad_norm,▂▂█▁
train/learning_rate,█▆▃▁
train/loss,█▄▂▁


In [23]:
final_trainer_bert.save_model(f"models/bert_final_stratify_{is_preprocessed}")
bert_tokenizer.save_pretrained(f"models/bert_final_stratify_{is_preprocessed}")
!cp -r models/bert_final_stratify_{is_preprocessed} "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/w_test_split/bert_best_model_stratify_{is_preprocessed}"

final_trainer_roberta.save_model(f"models/roberta_final_stratify_{is_preprocessed}")
roberta_tokenizer.save_pretrained(f"models/roberta_final_stratify_{is_preprocessed}")
!cp -r models/roberta_final_stratify_{is_preprocessed} "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/w_test_split/roberta_best_model_stratify_{is_preprocessed}"